In [3]:
filename = 'points_NSX_TMD_16_25_17_5.json'
In = 17.5


import numpy as np
import json

def get_relay_curve(In=1, order=4, C=1):
    def relay_curve(I, *ks):
        t = 0
        for order, k in enumerate(ks):
            t += k*np.log10((I/In)-C)**order
        return 10**t
    return relay_curve

with open(filename) as f:
    data = json.load(f)['Points']

x = np.array([row['X'] for row in data])/In
y = np.array([row['Y'] for row in data])
points_n = np.argwhere(x == np.min(x))[0,0]
i_l = x[3:points_n][::-1]
t_l = y[3:points_n][::-1]
i_u = x[points_n+3:]
t_u = y[points_n+3:]

l_filter = t_l <= 3600*2
u_filter = t_u <= 3600*2
i_l = i_l[l_filter]
t_l = t_l[l_filter]
i_u = i_u[u_filter]
t_u = t_u[u_filter]

def reduce(i, t):
    i1 = i[0]
    i2 = i[-1]
    i_n = np.geomspace(i1,i2,num=50)
    i_n_nd = np.tile(i_n, (len(i), 1))
    i_nd = np.tile(i, (len(i_n), 1)).T
    idx = (np.abs(i_n_nd - i_nd)).argmin(axis=0)
    return i[idx], t[idx]

i_u, t_u = reduce(i_u, t_u)
i_l, t_l = reduce(i_l, t_l)

In [4]:
import matplotlib.pyplot as plt
%matplotlib auto

plt.plot(i_u, t_u, 'b-', label='upper')
plt.plot(i_l, t_l, 'r-', label='lower')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,100)
plt.ylim(0.1,1e4)
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()

Using matplotlib backend: TkAgg


In [12]:
# Curve fitting upper

In = 1.28
order=6
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_u)
ydata = np.array(t_u)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 25, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[1.79633, -1.55481, 0.06144, 0.12102, -0.26556, -0.02143, 0.06357]

In [9]:
# Curve fitting lower

In=1.03
order=7
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_l)
ydata = np.array(t_l)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 15, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[1.72193, -1.91199, 0.34472, 0.74576, -0.68031, -0.66469, 0.2778, 0.22385]

In [124]:
print(np.min(i_l), np.min(i_u))
print(json.dumps(list(np.round(i_u, 5))), '\n')
print(json.dumps(list(np.round(t_u, 5))), '\n')
print(json.dumps(list(np.round(i_l, 5))), '\n')
print(json.dumps(list(np.round(t_l, 5))), '\n')

1.057142857142857 1.307142857142857
[1.30714, 1.37857, 1.45, 1.52857, 1.61429, 1.7, 1.79286, 1.88571, 1.98571, 2.09286, 2.20714, 2.32857, 2.45, 2.58571, 2.72143, 2.87143, 3.02143, 3.18571, 3.35714, 3.54286, 3.72857, 3.92857, 4.14286, 4.36429, 4.6, 4.85, 5.10714, 5.38571, 5.67857, 5.97857, 6.3, 6.64286, 7.0, 7.35714, 7.78571, 8.21429, 8.64286, 9.07143, 9.57143, 10.07143, 10.64286, 11.21429, 11.85714, 12.42857, 13.14286, 13.85714, 14.57143, 15.35714, 16.21429, 17.07143] 

[6650.88944, 3914.53987, 2664.68369, 1978.62688, 1454.45198, 1122.96479, 898.14355, 726.48328, 584.73676, 468.91133, 385.70913, 320.11806, 268.21081, 222.32379, 191.08848, 164.28756, 142.9887, 125.4339, 110.17951, 96.4386, 85.88768, 76.30254, 67.65535, 60.12971, 53.71777, 48.00716, 43.27092, 39.22578, 35.56931, 32.34129, 29.35811, 26.61936, 24.16397, 22.04099, 19.85135, 17.9798, 16.36692, 14.96667, 13.55392, 12.33666, 11.14055, 10.11422, 9.12421, 8.36411, 7.54348, 6.84065, 6.23388, 5.6573, 5.11705, 4.65239] 

[1.05714, 